In [1]:
import os
import sys
import subprocess
from skimage import io
import numpy as np
import itk
# Add the 'src' folder to the Python path
sys.path.append(os.path.abspath('../'))
from src.utils import match_h5_files_by_channels, read_h5_image
from src.registration import register_and_transform
from src.stats import compute_region_stats

# single animal

In [2]:
def process_animal(animal, base_dir, fx, param_files, annotation_np):
    """
    Process a single animal by:
    - Registering and transforming the images
    - Computing regional statistics
    """
    animals = match_h5_files_by_channels(base_dir)
    files = animals.get(animal)
    
    if not files:
        print(f"No files found for animal {animal}")
        return

    output_dir = os.path.join(base_dir, animal, 'itk')

    # Step 1: Register and Transform
    register_and_transform(fx, files, output_dir, param_files)

    # Step 2: Compute Region Statistics
    compute_region_stats(files, output_dir, annotation_np)


In [ ]:
# I have a bunch of folders for a batch of animals that were imaged toghether
base_dir = '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains'

In [ ]:
# this is the atlas image and anotation image I use for registration (10um voxel size cut in half)
fx = itk.imread('/nrs/spruston/Boaz/I2/atlas10_hemi.tif', pixel_type=itk.US)
annotation_np = np.int64(io.imread('/nrs/spruston/Boaz/I2/annotatin10_hemi.tif'))

In [ ]:
# these are the registation paramaters, they are also in the repo under param files
param_files = [
    '/nrs/spruston/Boaz/I2/itk/Order1_Par0000affine.txt',
    '/nrs/spruston/Boaz/I2/itk/Order3_Par0000bspline.txt',
    '/nrs/spruston/Boaz/I2/itk/Order4_Par0000bspline.txt',
    '/nrs/spruston/Boaz/I2/itk/Order5_Par0000bspline.txt'
]


In [ ]:
#this is one of the subfolders from the base path, the output of the MuVi lightsheet
animal = 'ANM550749_left_JF552'
process_animal(animal, base_dir, fx, param_files, annotation_np)

## submit to cluster

In [ ]:
def submit_job(animal, base_dir, logs_dir, python_script):
    """
    Function to submit a job to the cluster using bsub
    :param animal: The animal ID to process
    :param base_dir: Base directory for the data
    :param logs_dir: Directory for log files
    :param python_script: Path to the Python script
    """
    job_name = f"job_{animal}"
    output_log = os.path.join(logs_dir, f"output_{animal}.log")
    error_log = os.path.join(logs_dir, f"error_{animal}.log")
    
    bsub_command = [
        'bsub',
        '-J', job_name,
        '-n', '4',  # Number of CPU cores
        '-W', '4:00',  # Wall-clock time limit
        '-R', 'rusage[mem=8000]',  # Memory request
        '-o', output_log,  # Output log
        '-e', error_log,  # Error log
        'python', python_script, '--animal', animal
    ]
    
    print(f"Submitting job for animal {animal}...")
    subprocess.run(bsub_command)
